In [1]:
import cv2
import time
import json
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

In [2]:
def video_read(video_path):
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    buf = np.empty((frame_count, frame_height, frame_width, 3), np.dtype('uint8'))
    fc = 0
    ret = True
    while fc < frame_count and ret:
        ret, frame = video.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            buf[fc] = frame
            fc += 1
    video.release()
    return fc, buf

In [3]:
def detect_faces(mtcnn, video):
    return [mtcnn.detect(Image.fromarray(frame)) for frame in video]

In [4]:
def get_face_info(segments):
    total_probability_threshold = 0.1
    faces = []
    def insert_info(info, frame_num, rect, proba):
        info["last_frame"] = frame_num
        info["widths"][frame_num] = int(rect[2] - rect[0])
        info["heights"][frame_num] = int(rect[3] - rect[1])
        info["centers_x"][frame_num] = int(rect[0] + (info["widths"][frame_num] / 2))
        info["centers_y"][frame_num] = int(rect[1] + (info["heights"][frame_num] / 2))
        info["probs"][frame_num] = int(proba * 100)
    for frame_num, frame_segment in enumerate(segments):
        if frame_segment[0] is not None:
            for rect, prob in zip(frame_segment[0], frame_segment[1]):
                matched = False
                for face_info in faces:
                    last_frame = face_info["last_frame"]
                    if last_frame < frame_num:
                        if rect[0] < face_info["centers_x"][last_frame] < rect[2] \
                        and rect[1] < face_info["centers_y"][last_frame] < rect[3]:
                            insert_info(face_info, frame_num, rect, prob)
                            matched = True
                            break
                if not matched:
                    face_info = {
                        "last_frame": 0,
                        "centers_x": [0] * len(segments),
                        "centers_y": [0] * len(segments),
                        "widths": [0] * len(segments),
                        "heights": [0] * len(segments),
                        "probs": [0] * len(segments)
                    }
                    faces.append(face_info)
                    insert_info(face_info, frame_num, rect, prob)
    faces = list(filter(lambda info: np.mean(info["probs"]) > total_probability_threshold * 100, faces))
    return faces[:min(len(faces), 5)]

In [5]:
def read_metadata(path):
    metadata_file = open(path)
    metadata = json.load(metadata_file)
    metadata_file.close()
    return metadata

In [6]:
def write_metadata(metadata, path):
    with open(path, 'w') as output:
        json.dump(metadata, output)

In [11]:
dir = "D:\\Projects\\DFDC\\Data\\"
source_metadata_file_name = 'metadata-processed.json'
output_dir = "C:\\Users\\yuval\\Documents\\"
output_metadata_file_name = 'metadata-processed.json'

metadata = read_metadata(dir + source_metadata_file_name)
real_count = len(list(filter(lambda info: info['label'] == 'REAL', metadata.values())))
print("Metadata contains {} videos, from which {} are real".format(
    len(metadata), real_count
))

Metadata contains 119154 videos, from which 19154 are real


In [ ]:
recalculate_function = lambda info: 'faces' not in info

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
mtcnn = MTCNN(keep_all=True, select_largest=False, post_process=False, min_face_size=30, device=device)

updated_count = 0
for video_name, info in metadata.items():
    if info['label'] == 'REAL':
        sub_dir = info['dir']
        if recalculate_function(info):
            print("Reading video - {}\\{}".format(sub_dir, video_name))
            t0 = time.time()
            fc, video = video_read("{}{}\\{}".format(dir, sub_dir, video_name))
            video = video[:fc]
            t1 = time.time()
            print("\t{} frames read in {}s. Detecting faces...".format(fc, t1 - t0))
            segments = detect_faces(mtcnn, video)
            t2 = time.time()
            print("\tFace segments found in {}s. Processing faces...".format(t2 - t1))
            faces = get_face_info(segments)
            t3 = time.time()
            print("\t{} faces processed in {}s. Rewriting {} ...".format(len(faces), t3 - t2, output_metadata_file_name))
            info['faces'] = faces
            if updated_count % 20 == 0:
                write_metadata(metadata, dir + output_metadata_file_name)
            t4 = time.time()
            print("\t{} was updated in {}s".format(output_metadata_file_name, t4 - t3))
            print("\tOverall time: {}s".format(t4 - t0))
        else:
            print("Video - {}\\{} faces already exist. Skipping".format(sub_dir, video_name))
        updated_count += 1
        print("\tUpdated metadata for {} out of {} original movie. {} more to go...".format(updated_count, real_count, real_count - updated_count))

Running on device: cuda:0
Video - dfdc_train_part_0\vpmyeepbep.mp4 faces already exist. Skipping
	Updated metadata for 1 out of 19154 original movie. 19153 more to go...
Video - dfdc_train_part_0\fzvpbrzssi.mp4 faces already exist. Skipping
	Updated metadata for 2 out of 19154 original movie. 19152 more to go...
Video - dfdc_train_part_0\syxobtuucp.mp4 faces already exist. Skipping
	Updated metadata for 3 out of 19154 original movie. 19151 more to go...
Video - dfdc_train_part_0\dhjnjkzuhq.mp4 faces already exist. Skipping
	Updated metadata for 4 out of 19154 original movie. 19150 more to go...
Video - dfdc_train_part_0\xcruhaccxc.mp4 faces already exist. Skipping
	Updated metadata for 5 out of 19154 original movie. 19149 more to go...
Video - dfdc_train_part_0\vtunvalyji.mp4 faces already exist. Skipping
	Updated metadata for 6 out of 19154 original movie. 19148 more to go...
Video - dfdc_train_part_0\qyqufaskjs.mp4 faces already exist. Skipping
	Updated metadata for 7 out of 19154 or

	Updated metadata for 646 out of 19154 original movie. 18508 more to go...
Video - dfdc_train_part_11\kmnboeohyo.mp4 faces already exist. Skipping
	Updated metadata for 647 out of 19154 original movie. 18507 more to go...
Video - dfdc_train_part_11\jfrkmgldsg.mp4 faces already exist. Skipping
	Updated metadata for 648 out of 19154 original movie. 18506 more to go...
Video - dfdc_train_part_11\jxjahoczxz.mp4 faces already exist. Skipping
	Updated metadata for 649 out of 19154 original movie. 18505 more to go...
Video - dfdc_train_part_11\gzukcthgqg.mp4 faces already exist. Skipping
	Updated metadata for 650 out of 19154 original movie. 18504 more to go...
Video - dfdc_train_part_11\rzekdyfhks.mp4 faces already exist. Skipping
	Updated metadata for 651 out of 19154 original movie. 18503 more to go...
Video - dfdc_train_part_11\nxheolhgom.mp4 faces already exist. Skipping
	Updated metadata for 652 out of 19154 original movie. 18502 more to go...
Video - dfdc_train_part_11\csbzqhunqw.mp4 f

Video - dfdc_train_part_12\kxzyciuuqa.mp4 faces already exist. Skipping
	Updated metadata for 1167 out of 19154 original movie. 17987 more to go...
Video - dfdc_train_part_12\ikaguhjtav.mp4 faces already exist. Skipping
	Updated metadata for 1168 out of 19154 original movie. 17986 more to go...
Video - dfdc_train_part_12\mkpxlnvlil.mp4 faces already exist. Skipping
	Updated metadata for 1169 out of 19154 original movie. 17985 more to go...
Video - dfdc_train_part_12\cvotzhernt.mp4 faces already exist. Skipping
	Updated metadata for 1170 out of 19154 original movie. 17984 more to go...
Video - dfdc_train_part_12\icnrizpsic.mp4 faces already exist. Skipping
	Updated metadata for 1171 out of 19154 original movie. 17983 more to go...
Video - dfdc_train_part_12\rwrxltmsto.mp4 faces already exist. Skipping
	Updated metadata for 1172 out of 19154 original movie. 17982 more to go...
Video - dfdc_train_part_12\udhoamgxmx.mp4 faces already exist. Skipping
	Updated metadata for 1173 out of 19154 

Video - dfdc_train_part_14\gdavsauheg.mp4 faces already exist. Skipping
	Updated metadata for 1757 out of 19154 original movie. 17397 more to go...
Video - dfdc_train_part_14\sjhtuovugl.mp4 faces already exist. Skipping
	Updated metadata for 1758 out of 19154 original movie. 17396 more to go...
Video - dfdc_train_part_14\vjnmgqnbkm.mp4 faces already exist. Skipping
	Updated metadata for 1759 out of 19154 original movie. 17395 more to go...
Video - dfdc_train_part_14\blzgjrjsvy.mp4 faces already exist. Skipping
	Updated metadata for 1760 out of 19154 original movie. 17394 more to go...
Video - dfdc_train_part_14\tqgjmwzsij.mp4 faces already exist. Skipping
	Updated metadata for 1761 out of 19154 original movie. 17393 more to go...
Video - dfdc_train_part_14\tctuytrzad.mp4 faces already exist. Skipping
	Updated metadata for 1762 out of 19154 original movie. 17392 more to go...
Video - dfdc_train_part_14\zyicfdblvi.mp4 faces already exist. Skipping
	Updated metadata for 1763 out of 19154 

	Updated metadata for 2475 out of 19154 original movie. 16679 more to go...
Video - dfdc_train_part_16\qaehvzuaga.mp4 faces already exist. Skipping
	Updated metadata for 2476 out of 19154 original movie. 16678 more to go...
Video - dfdc_train_part_16\wuhhueyezm.mp4 faces already exist. Skipping
	Updated metadata for 2477 out of 19154 original movie. 16677 more to go...
Video - dfdc_train_part_16\vqaptjacdp.mp4 faces already exist. Skipping
	Updated metadata for 2478 out of 19154 original movie. 16676 more to go...
Video - dfdc_train_part_16\skgojnlrnp.mp4 faces already exist. Skipping
	Updated metadata for 2479 out of 19154 original movie. 16675 more to go...
Video - dfdc_train_part_16\rvnsxxxcde.mp4 faces already exist. Skipping
	Updated metadata for 2480 out of 19154 original movie. 16674 more to go...
Video - dfdc_train_part_16\axnpeqmffl.mp4 faces already exist. Skipping
	Updated metadata for 2481 out of 19154 original movie. 16673 more to go...
Video - dfdc_train_part_16\jfzjnmnar

...

	300 frames read in 5.293747186660767s. Detecting faces...
	Face segments found in 26.354793548583984s. Processing faces...
	1 faces processed in 0.005983591079711914s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 31.654524326324463s
	Updated metadata for 16498 out of 19154 original movie. 2656 more to go...
Reading video - dfdc_train_part_47\losdmmgngh.mp4
	300 frames read in 5.317986965179443s. Detecting faces...
	Face segments found in 26.28956389427185s. Processing faces...
	1 faces processed in 0.0059833526611328125s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 31.613534212112427s
	Updated metadata for 16499 out of 19154 original movie. 2655 more to go...
Reading video - dfdc_train_part_47\ppghcvwxhb.mp4
	300 frames read in 5.28824782371521s. Detecting faces...
	Face segments found in 23.634018182754517s. Processing faces...
	1 faces processed in 0.005983829498291016s. Rewrit

	301 frames read in 2.748065710067749s. Detecting faces...
	Face segments found in 23.36874747276306s. Processing faces...
	1 faces processed in 0.0069806575775146484s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 26.123793840408325s
	Updated metadata for 16518 out of 19154 original movie. 2636 more to go...
Reading video - dfdc_train_part_47\snjcinquse.mp4
	301 frames read in 2.684744358062744s. Detecting faces...
	Face segments found in 24.499263048171997s. Processing faces...
	1 faces processed in 0.009973764419555664s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 27.193981170654297s
	Updated metadata for 16519 out of 19154 original movie. 2635 more to go...
Reading video - dfdc_train_part_47\ulbvjauchz.mp4
	301 frames read in 2.6043713092803955s. Detecting faces...
	Face segments found in 29.632326126098633s. Processing faces...
	1 faces processed in 0.011967658996582031s. Rewr

	300 frames read in 5.060055732727051s. Detecting faces...
	Face segments found in 22.91644811630249s. Processing faces...
	1 faces processed in 0.006982088088989258s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 27.98348593711853s
	Updated metadata for 16538 out of 19154 original movie. 2616 more to go...
Reading video - dfdc_train_part_48\ofapaanpkl.mp4
	300 frames read in 5.3500330448150635s. Detecting faces...
	Face segments found in 27.097683668136597s. Processing faces...
	1 faces processed in 0.007979154586791992s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 32.45569586753845s
	Updated metadata for 16539 out of 19154 original movie. 2615 more to go...
Reading video - dfdc_train_part_48\gsxlolioxh.mp4
	300 frames read in 5.147470712661743s. Detecting faces...
	Face segments found in 24.553022861480713s. Processing faces...
	1 faces processed in 0.007978677749633789s. Rewriti

	300 frames read in 5.304524660110474s. Detecting faces...
	Face segments found in 23.57610559463501s. Processing faces...
	1 faces processed in 0.0069811344146728516s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 28.887611389160156s
	Updated metadata for 16558 out of 19154 original movie. 2596 more to go...
Reading video - dfdc_train_part_48\mhvebxvhsx.mp4
	300 frames read in 5.410847902297974s. Detecting faces...
	Face segments found in 21.370431423187256s. Processing faces...
	1 faces processed in 0.005983829498291016s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 26.78726315498352s
	Updated metadata for 16559 out of 19154 original movie. 2595 more to go...
Reading video - dfdc_train_part_48\xsxwrwvfkg.mp4
	300 frames read in 5.476002931594849s. Detecting faces...
	Face segments found in 26.085925579071045s. Processing faces...
	1 faces processed in 0.005984067916870117s. Rewrit

	300 frames read in 5.315664529800415s. Detecting faces...
	Face segments found in 21.660643339157104s. Processing faces...
	1 faces processed in 0.006981372833251953s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 26.98328924179077s
	Updated metadata for 16578 out of 19154 original movie. 2576 more to go...
Reading video - dfdc_train_part_48\rwsvyxrubl.mp4
	300 frames read in 5.331714868545532s. Detecting faces...
	Face segments found in 21.213505029678345s. Processing faces...
	1 faces processed in 0.005984067916870117s. Rewriting metadata-processed.json ...
	metadata-processed.json was updated in 0.0s
	Overall time: 26.551203966140747s
	Updated metadata for 16579 out of 19154 original movie. 2575 more to go...
Reading video - dfdc_train_part_48\vdqritvjfl.mp4
	300 frames read in 5.272943735122681s. Detecting faces...
	Face segments found in 22.599159479141235s. Processing faces...
	1 faces processed in 0.005983829498291016s. Rewrit